In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('countypres_2000-2020.csv')
df.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL


In [3]:
county_df = df[['year', 'state_po', 'county_name', 'county_fips', 'totalvotes']].drop_duplicates()
county_df.head()

,year,state_po,county_name,county_fips,totalvotes
0,2000,AL,AUTAUGA,1001.0,17208
4,2000,AL,BALDWIN,1003.0,56480
8,2000,AL,BARBOUR,1005.0,10395
12,2000,AL,BIBB,1007.0,7101
16,2000,AL,BLOUNT,1009.0,17973


In [4]:
county_df = county_df.drop(columns='dvotes', errors='ignore').merge(df[df["party"] == "DEMOCRAT"][['year', 'state_po', 'county_fips', 'candidatevotes']],
                            on=['year', 'state_po', 'county_fips'],
                            how='left').rename(columns={'candidatevotes':'dvotes'})
county_df = county_df.drop(columns='rvotes', errors='ignore').merge(df[df["party"] == "REPUBLICAN"][['year', 'state_po', 'county_fips', 'candidatevotes']],
                            on=['year', 'state_po', 'county_fips'],
                            how='left').rename(columns={'candidatevotes':'rvotes'})
county_df.head()

,year,state_po,county_name,county_fips,totalvotes,dvotes,rvotes
0,2000,AL,AUTAUGA,1001.0,17208,4942,11993
1,2000,AL,BALDWIN,1003.0,56480,13997,40872
2,2000,AL,BARBOUR,1005.0,10395,5188,5096
3,2000,AL,BIBB,1007.0,7101,2710,4273
4,2000,AL,BLOUNT,1009.0,17973,4977,12667


In [5]:
state_df = county_df[['year', 'state_po']].drop_duplicates()

In [6]:
totalvotes = county_df.groupby(['year', 'state_po'])['totalvotes'].sum()
county_df = county_df.merge(totalvotes, on=['year', 'state_po'], how='left').rename(columns={'totalvotes_x':'totalvotes', 'totalvotes_y':'statevotes'})
county_df.head()

,year,state_po,county_name,county_fips,totalvotes,dvotes,rvotes,statevotes
0,2000,AL,AUTAUGA,1001.0,17208,4942,11993,1672551
1,2000,AL,BALDWIN,1003.0,56480,13997,40872,1672551
2,2000,AL,BARBOUR,1005.0,10395,5188,5096,1672551
3,2000,AL,BIBB,1007.0,7101,2710,4273,1672551
4,2000,AL,BLOUNT,1009.0,17973,4977,12667,1672551


In [10]:
county_df['vote_percentage'] = county_df.apply(lambda x: x['totalvotes'] / x['statevotes'], axis=1)
county_df['r_margin_perc'] = county_df.apply(lambda x: (x['rvotes'] - x['dvotes']) / x['statevotes'], axis=1)
county_df.head()

,year,state_po,county_name,county_fips,totalvotes,dvotes,rvotes,statevotes,vote_percentage,r_margin_perc
0,2000,AL,AUTAUGA,1001.0,17208,4942,11993,1672551,0.010288,0.004216
1,2000,AL,BALDWIN,1003.0,56480,13997,40872,1672551,0.033769,0.016068
2,2000,AL,BARBOUR,1005.0,10395,5188,5096,1672551,0.006215,-0.000055
3,2000,AL,BIBB,1007.0,7101,2710,4273,1672551,0.004246,0.000935
4,2000,AL,BLOUNT,1009.0,17973,4977,12667,1672551,0.010746,0.004598


In [ ]:
df.to_csv('../site/data/county_data.csv', index=False)